In [1]:
# online is "hf://datasets/princeton-nlp/SWE-bench/"
# local is "~/SWE-bench"

# DATASET_PATH = "~/SWE-bench"
DATASET_PATH = "hf://datasets/princeton-nlp/SWE-bench/"

import pandas as pd

splits = {
    "dev": "data/dev-00000-of-00001.parquet",
    "test": "data/test-00000-of-00001.parquet",
    "train": "data/train-00000-of-00001.parquet",
}
df = pd.read_parquet(DATASET_PATH + splits["dev"])
print(f"We have {len(df)} rows in the dev set")
print(df.head())
print("------")

# print one example
print(df.iloc[0])

/Users/zen.tang/swe-bench-solver/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We have 225 rows in the dev set
                repo              instance_id  \
0  sqlfluff/sqlfluff  sqlfluff__sqlfluff-4764   
1  sqlfluff/sqlfluff  sqlfluff__sqlfluff-2862   
2  sqlfluff/sqlfluff  sqlfluff__sqlfluff-2336   
3  sqlfluff/sqlfluff  sqlfluff__sqlfluff-5074   
4  sqlfluff/sqlfluff  sqlfluff__sqlfluff-3436   

                                base_commit  \
0  a820c139ccbe6d1865d73c4a459945cd69899f8f   
1  447ecf862a4d2b977d0add9f444655357b9c4f1f   
2  37a993f7ad841ab3035d1db5ce6525f2e5584fd5   
3  7b7fd603a19755a9f3707ebbf95d18ee635716d8   
4  23cd31e77a712a210c734e38488d7a34afd83a25   

                                               patch  \
0  diff --git a/src/sqlfluff/cli/commands.py b/sr...   
1  diff --git a/src/sqlfluff/core/linter/common.p...   
2  diff --git a/src/sqlfluff/core/rules/analysis/...   
3  diff --git a/src/sqlfluff/core/errors.py b/src...   
4  diff --git a/src/sqlfluff/core/templaters/slic...   

                                          test_patch 

In [7]:
row = df.iloc[0]
dict(row)


{'repo': 'sqlfluff/sqlfluff',
 'instance_id': 'sqlfluff__sqlfluff-4764',
 'base_commit': 'a820c139ccbe6d1865d73c4a459945cd69899f8f',
 'patch': 'diff --git a/src/sqlfluff/cli/commands.py b/src/sqlfluff/cli/commands.py\n--- a/src/sqlfluff/cli/commands.py\n+++ b/src/sqlfluff/cli/commands.py\n@@ -44,6 +44,7 @@\n     dialect_selector,\n     dialect_readout,\n )\n+from sqlfluff.core.linter import LintingResult\n from sqlfluff.core.config import progress_bar_configuration\n \n from sqlfluff.core.enums import FormatType, Color\n@@ -691,12 +692,16 @@ def lint(\n         sys.exit(EXIT_SUCCESS)\n \n \n-def do_fixes(lnt, result, formatter=None, **kwargs):\n+def do_fixes(\n+    result: LintingResult, formatter: Optional[OutputStreamFormatter] = None, **kwargs\n+):\n     """Actually do the fixes."""\n-    click.echo("Persisting Changes...")\n+    if formatter and formatter.verbosity >= 0:\n+        click.echo("Persisting Changes...")\n     res = result.persist_changes(formatter=formatter, **kwargs)\

In [22]:
def visualize_row(row):
    print("------ REPO ------")
    print(row["repo"])
    print("------ CREATED AT ------")
    print(row["created_at"])
    if row["hints_text"]:
        print("------ HINTS ------")
        print(row["hints_text"])
    print("------ PROBLEM STATEMENT ------")
    print(row["problem_statement"])
    print("------ PATCH ------")
    print(row["patch"])
    print("------ TESTS ------")
    print(row["test_patch"])

from itertools import islice
for i, row in islice(df.iterrows(), 200):
    # visualize_row(row)
    print(f"-------------PROBLEM STATEMENT: {row['repo']}----------------")
    print(f"-------------Fix: {len(row['patch'].split('\n'))} lines----------------")
    print(row["problem_statement"])

-------------PROBLEM STATEMENT: sqlfluff/sqlfluff----------------
-------------Fix: 308 lines----------------
Enable quiet mode/no-verbose in CLI for use in pre-commit hook
There seems to be only an option to increase the level of verbosity when using SQLFluff [CLI](https://docs.sqlfluff.com/en/stable/cli.html), not to limit it further.

It would be great to have an option to further limit the amount of prints when running `sqlfluff fix`, especially in combination with deployment using a pre-commit hook. For example, only print the return status and the number of fixes applied, similar to how it is when using `black` in a pre-commit hook:
![image](https://user-images.githubusercontent.com/10177212/140480676-dc98d00b-4383-44f2-bb90-3301a6eedec2.png)

This hides the potentially long list of fixes that are being applied to the SQL files, which can get quite verbose.

-------------PROBLEM STATEMENT: sqlfluff/sqlfluff----------------
-------------Fix: 359 lines----------------
fix keep addi

In [24]:
for k,v in df.iloc[199].items():
    print(f"{k}: {type(v)}")

repo: <class 'str'>
instance_id: <class 'str'>
base_commit: <class 'str'>
patch: <class 'str'>
test_patch: <class 'str'>
problem_statement: <class 'str'>
hints_text: <class 'str'>
created_at: <class 'str'>
version: <class 'str'>
FAIL_TO_PASS: <class 'str'>
PASS_TO_PASS: <class 'str'>
environment_setup_commit: <class 'str'>


In [21]:
import os
from dataclasses import dataclass
from typing import List, Optional

from swe_bench_solver.ls import files_list

@dataclass
class File:
    name: str
    fullpath: str
    size: int
    n_lines: int
    type: str

code_dir = os.path.expanduser("~/SWE-bench")
file_list = files_list(code_dir)

for file in file_list:
    print(f"{file.name.ljust(30)} ({file.type.ljust(10)}) has {file.n_lines} lines")

requirements-dev.lock          (lock      ) has 138 lines
codecov.yml                    (yml       ) has 16 lines
LICENSE                        (          ) has 21 lines
CHANGELOG.md                   (md        ) has 111 lines
pyproject.toml                 (toml      ) has 62 lines
README.md                      (md        ) has 143 lines
requirements.lock              (lock      ) has 138 lines
test_evaluation.py             (py        ) has 35 lines
test_collect_cli.py            (py        ) has 27 lines
test_cli.py                    (py        ) has 17 lines
README_CN.md                   (md        ) has 89 lines
README_JP.md                   (md        ) has 90 lines
README_TW.md                   (md        ) has 89 lines
README.md                      (md        ) has 76 lines
update_swe_bench.ipynb         (ipynb     ) has 632 lines
README.md                      (md        ) has 155 lines
sweep_conda_links.py           (py        ) has 61 lines
check_harness.ipynb      